# Training ResNet and SparseResNet on clean images 
### Flatten ReLU

In [1]:
import sys, os
d = os.path.dirname(os.getcwd())
sys.path.insert(0, d)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader

from utils.train_utils import train_model
from models.resnet_FReLU import FResNet, FSparseResNet
from utils.attacks import pgd

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [4]:
tr_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
vl_transform = transforms.Compose([
    transforms.ToTensor()
])

ds = CIFAR10('../data', train=True, download=True, transform=tr_transform)
ds_test = CIFAR10('../data', train=False, download=True, transform=vl_transform)

batch_size = 200
train_dl = DataLoader(ds, batch_size, shuffle=True)
valid_dl = DataLoader(ds_test, batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


# Training on clean images

### ResNet

In [7]:
model = FResNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', min_lr=1e-7, patience=2, verbose=True)

train_model(model, train_dl, valid_dl, optimizer, loss_fn, scheduler, epochs=30, device=device)
torch.save(model.state_dict(), "../saved/resnet_FReLU.pt")

Epoch: 1 Validation Loss: 1.1039 accuracy: 0.5956, time: 0:02:19
Epoch: 2 Validation Loss: 0.8268 accuracy: 0.7065, time: 0:02:23
Epoch: 3 Validation Loss: 0.7283 accuracy: 0.7433, time: 0:02:18
Epoch: 4 Validation Loss: 0.7232 accuracy: 0.7465, time: 0:02:19
Epoch: 5 Validation Loss: 0.5869 accuracy: 0.8024, time: 0:02:19
Epoch: 6 Validation Loss: 0.5537 accuracy: 0.8105, time: 0:02:19
Epoch: 7 Validation Loss: 0.5566 accuracy: 0.8074, time: 0:02:18
epochs_no_improve: 1/4
Epoch: 8 Validation Loss: 0.5200 accuracy: 0.8280, time: 0:02:17
Epoch: 9 Validation Loss: 0.4764 accuracy: 0.8396, time: 0:02:16
Epoch: 10 Validation Loss: 0.4890 accuracy: 0.8398, time: 0:02:15
epochs_no_improve: 1/4
Epoch: 11 Validation Loss: 0.4348 accuracy: 0.8583, time: 0:02:16
Epoch: 12 Validation Loss: 0.4208 accuracy: 0.8614, time: 0:02:18
Epoch: 13 Validation Loss: 0.4170 accuracy: 0.8647, time: 0:02:19
Epoch: 14 Validation Loss: 0.4214 accuracy: 0.8594, time: 0:02:15
epochs_no_improve: 1/4
Epoch: 15 Valida

### SparseResNet

In [5]:
model = FSparseResNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', min_lr=1e-7, patience=2, verbose=True)

train_model(model, train_dl, valid_dl, optimizer, loss_fn, scheduler, epochs=30, sparse=True, device=device)
torch.save(model.state_dict(), "../saved/sparse_resnet_FReLU.pt")

Epoch: 1 Validation Loss: 1.5932 accuracy: 0.3721, time: 0:02:39
Epoch: 2 Validation Loss: 1.4145 accuracy: 0.4425, time: 0:02:30
Epoch: 3 Validation Loss: 1.2446 accuracy: 0.5448, time: 0:02:33
Epoch: 4 Validation Loss: 1.0377 accuracy: 0.6287, time: 0:02:34
Epoch: 5 Validation Loss: 0.9092 accuracy: 0.6794, time: 0:02:33
Epoch: 6 Validation Loss: 0.8423 accuracy: 0.7059, time: 0:02:34
Epoch: 7 Validation Loss: 0.7903 accuracy: 0.7273, time: 0:02:33
Epoch: 8 Validation Loss: 0.7280 accuracy: 0.7466, time: 0:02:37
Epoch: 9 Validation Loss: 0.6973 accuracy: 0.7609, time: 0:02:37
Epoch: 10 Validation Loss: 0.6496 accuracy: 0.7779, time: 0:02:36
Epoch: 11 Validation Loss: 0.6335 accuracy: 0.7853, time: 0:02:35
Epoch: 12 Validation Loss: 0.6115 accuracy: 0.7957, time: 0:02:36
Epoch: 13 Validation Loss: 0.5929 accuracy: 0.7993, time: 0:02:34
Epoch: 14 Validation Loss: 0.5901 accuracy: 0.8050, time: 0:02:35
Epoch: 15 Validation Loss: 0.5720 accuracy: 0.8096, time: 0:02:34
Epoch: 16 Validatio